### Packages

In [1]:
import numpy as np
import pandas as pd
import requests
import os
from tqdm import tqdm
from sqlalchemy import create_engine

### Clear documents

In [2]:
if os.path.exists("Journal.xlsx"):
    os.remove("Journal.xlsx")
    
if os.path.exists("Crypto.db"):
    os.remove("Crypto.db")

### Input

In [3]:
coins = ('BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'SOLUSDT', 'ADAUSDT', 'DOTUSDT', 'DOGEUSDT', 'AVAXUSDT', 'SHIBUSDT',
         'MATICUSDT', 'LTCUSDT', 'UNIUSDT', 'ALGOUSDT', 'TRXUSDT', 'LINKUSDT', 'MANAUSDT', 'ATOMUSDT', 'VETUSDT', 'FTMUSDT')

capital = 100
UTC = 7
SMA = 28 #Simple Moving Average
timeframe = '4h'
tradingfee = 0
#backtesting = Backtest_SMA(capital, simulation_time, pair, location, SMA)

### Download data

In [4]:
def download_data(symbol, timeframe):
    
    resp = requests.get('https://api.binance.us/api/v3/klines?symbol=' + symbol + '&interval=' + timeframe + '&limit=1000')
    frame = pd.DataFrame(resp.json())
    frame = frame.iloc[:,:5]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close']
    frame[['Open', 'High', 'Low', 'Close']] = frame[['Open', 'High', 'Low', 'Close']].astype(float)
    frame.Time = pd.to_datetime(frame.Time, unit='ms') + pd.Timedelta(hours=UTC)
    return frame

In [5]:
engine = create_engine('sqlite:///Crypto.db')

for coin in tqdm(coins):
    download_data(coin, timeframe).to_sql(coin, engine, index=False)

100%|██████████| 19/19 [00:22<00:00,  1.17s/it]


### Sort coin with high return from backtest

In [6]:
def technicals(df):
    df = df.copy()
    df['SMA'] = df.Close.rolling(SMA).mean()
    df.dropna(inplace=True)
    df['change'] = df.Close.pct_change()
        
    df['long strategy']  = np.where(df['Close'] > df['SMA'], 1, 0)
    df['Long positions'] = capital*(np.cumprod(df['long strategy'].shift(1)*df['change'] + 1))
        
    df['short strategy']  = np.where(df['Close'] > df['SMA'], 0, -1)
    df['Short positions'] = capital*(np.cumprod(df['short strategy'].shift(1)*df['change'] + 1))
        
    df['Return total'] = df['Long positions']*df['Short positions']/capital
    df['Return hold']  = capital*np.cumprod(df['change']+1)
    
    df.dropna(inplace=True)

    return df[['Return total', 'Return hold']]

In [7]:
profits = []
journal = pd.DataFrame()

for coin in coins:
    df = pd.read_sql(coin, engine).set_index('Time')
    ret = technicals(df)
    journal[coin] = ret.loc[:,'Return total']
    profits.append(ret.iloc[-1,:])


frame = pd.DataFrame(profits, coins)
frame['edges'] = frame['Return total'] - frame['Return hold']
frame = frame.sort_values('edges', ascending=False)
frame

,Return total,Return hold,edges
AVAXUSDT,155.071867,72.570533,82.501334
ETHUSDT,151.194777,80.213809,70.980967
SHIBUSDT,145.605701,103.580786,42.024915
BTCUSDT,91.369736,68.343063,23.026673
SOLUSDT,75.786166,68.447827,7.338339
BNBUSDT,109.272790,105.547017,3.725773
MATICUSDT,178.905995,175.748031,3.157964
ADAUSDT,81.404096,80.475766,0.928331
FTMUSDT,63.069046,71.401769,-8.332723
DOTUSDT,58.577051,67.761807,-9.184756


### Journal excel

In [8]:
journal.to_excel('Journal.xlsx', engine='xlsxwriter') 